In [1]:
import numpy as np
import json
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
from IPython.display import display

In [2]:
with open('data/kaggle/train.json') as f:
    data = json.load(f)
with open('nationalities.csv', 'r') as f:
    reader = csv.reader(f)
    nationalities = [item.lower() for sublist in list(reader)[1:] for item in sublist]
with open('to_remove.csv', 'r') as f:
    reader = csv.reader(f)
    to_remove = [item.lower() for sublist in list(reader) for item in sublist]

In [3]:
baseURL = "https://en.wikipedia.org/wiki/"
baseURL2 = "http://foodb.ca/unearth/q?searcher=foods&query="

In [4]:
ingr_size = list(['small', 'medium', 'large', 'big', 'flat'])

ingr_parts = list(['breast','breasts', 'tender', 'tenders', 'ribs', 'shoulder', 'fillet',
                   'steak', 'tigh', 'liver', 'livers',
                   'clove', 'cloves', 'leaf', 'leaves', 'flakes', 'threads', 'paste',
                   'piece', 'pieces', 'slice', 'slices', 'crumbles', 'powder', 'blend', 'mix'])

ingr_state = list(['grilled', 'roasted', 'toasted', 'fried', 'baked', 'cooked',
              'chopped', 'cut', 'sliced', 'shredded',
              'frozen', 'refrigerated', 'fresh', 'freshly', 'organic', 'natural', 'active',
              'boneless', 'skinless', 'condensed', 'ground', 'warm', 'cold'])

ingr_color = list(['purple', 'yellow', 'blue', 'red', 'white', 'black', 'green', 'brown'])

#to_remove_step1 = ingr_size+ingr_parts+ingr_state
to_remove_step2 = ingr_color



to_remove_step1  = list(['crumbles', 'ground', 'grilled', 'roasted', 'toasted', 'fried', 'baked', 'chopped',
                         'frozen', 'refrigerated', 'boneless', 'natural', 'organic', 'fresh', 'freshly',
                         'medium', 'large','big', 'small', 'breasts', 'tender', 'ribs', 'shoulder', 'paste', 'cloves', 'slivered',
                         'condensed', 'cut', 'piece', 'pieces', 'sliced', 'cooked', 'grated'])
to_remove_step2  = list(['purple', 'yellow', 'blue', 'red', 'white', 'black', 'green', 'dried'])

In [5]:
"""def try_find_ingredient2(ingr_list):
    ingr = '+'.join(ingr_list)
    str1 = "Went to foodDB for: "+ingr+" -> "
    try:
        url = urlopen(baseURL2+ingr)
    except:
        return ""
    res = BeautifulSoup(url.read(),"html5lib");
    title = res.find(class_= "hit-name").text.lower()
    title = re.sub(r" \(.*\)","", title)
    join = [value for value in title.split() if value in ingr_list]
    if len(join) > 0:
        title = ' '.join(join)
    print(str1+title)
    return title"""

def try_find_ingredient(ingr_list):
    ingr = '_'.join(ingr_list)
    try:
        url = urlopen(baseURL+ingr)
    except:
        return ""
    res = BeautifulSoup(url.read(),"html5lib");
    title = res.find(id="firstHeading").text.lower()
    title = re.sub(r" \(.*\)","", title) #remove (desambiguation)
    return title

In [6]:
def clean_ingredient(ingr):
    ingr = ingr.split(',')[0].split('(')[0].replace('%', "").lower()
    ingredient_list = [ing for ing in ingr.split() if (ing not in nationalities and not ing.isdigit()) ]
    ingredient = try_find_ingredient(ingredient_list)
    if ingredient == "":
        ingredient_list = [word for word in ingredient_list if (word not in to_remove)]
        ingredient = try_find_ingredient(ingredient_list)
    if ingredient == "":
        ingredient = ' '.join(ingredient_list)
    return ingredient

In [7]:
def get_ingredients_from_recipe(recipe):
    ingredients = set()
    for ingr in recipe['ingredients']:
        cleaned_ingr = clean_ingredient(ingr)
        if cleaned_ingr:
            ingredients.add(cleaned_ingr)
    return list(ingredients)

In [8]:
def print_ingredients_from_recipe(recipe):
    table = list()
    for ingr in recipe['ingredients']:
        cleaned_ingr = clean_ingredient(ingr)
        table.append((ingr, cleaned_ingr))
    df = pd.DataFrame.from_records(table, columns=['from', 'to'])
    display(df)

In [9]:
len(data)

39774

In [10]:
table = list()
for i in range(100,110):
    for ingr in data[i]['ingredients']:
        cleaned_ingr = clean_ingredient(ingr)
        table.append((ingr, cleaned_ingr))
df = pd.DataFrame.from_records(table, columns=['from', 'to'])
display(df)

from                           to
0                         tomato sauce                 tomato sauce
1                     shredded carrots                       carrot
2                              spinach                      spinach
3          part-skim mozzarella cheese  part-skim mozzarella cheese
4                    italian seasoning                    seasoning
5   english muffins, split and toasted                       muffin
6                        chopped onion                        onion
7          vegetable oil cooking spray  vegetable oil cooking spray
8            chopped green bell pepper                  bell pepper
9                               prawns                        prawn
10                          rice flour                   rice flour
11                      seasoning salt                seasoned salt
12                                salt                         salt
13                       vegetable oil                vegetable oil
14                           club soda                    club soda
15                       garlic powder                garlic powder
16                      cayenne pepper               cayenne pepper
17                      cheddar cheese               cheddar cheese
18                     lasagna noodles              lasagna noodles
19                              pepper                       pepper
20                      ranch dressing               ranch dressing
21                   mozzarella cheese                   mozzarella
22                      cooked chicken                      chicken
23                     evaporated milk              evaporated milk
24                       minced garlic                       garlic
25                       dried oregano                      oregano
26                    red wine vinegar                      vinegar
27                           olive oil                    olive oil
28                  boneless chop pork                    chop pork
29                         lemon juice                        lemon
..                                 ...                          ...
68           chopped green bell pepper                  bell pepper
69                   old bay seasoning            old bay seasoning
70                    dry bread crumbs                 bread crumbs
71         vegetable oil cooking spray  vegetable oil cooking spray
72                      golden raisins               golden raisins
73                                salt                         salt
74                           margarine                    margarine
75                                eggs                          egg
76                                mace                         mace
77                          brewed tea                   brewed tea
78                   all-purpose flour                        flour
79                 candied orange peel          candied orange peel
80                       ground nutmeg                       nutmeg
81                             raisins                       raisin
82                   grated lemon zest                         zest
83                     ground cinnamon                     cinnamon
84                      dried currants                      currant
85                       baking powder                baking powder
86                         rum extract                  rum extract
87                    dark brown sugar             dark brown sugar
88                               sugar                        sugar
89                               water                        water
90                             noodles                       noodle
91                                pork                         pork
92                         corn starch                  corn starch
93                       baby bok choy                     bok choy
94                    black bean sauce                       douchi
95                    

In [11]:
issues: jalapeno chilies -> chives

SyntaxError: invalid syntax (<ipython-input-11-1a7b3fb40c95>, line 1)